In [33]:
import chromadb
#https://github.com/chroma-core/chroma/blob/main/chromadb/experimental/density_relevance.ipynb

client =  chromadb.PersistentClient('./chroma_docs')
col_raw = client.get_collection('documents_raw')
#col_raw = client.get_collection('documents_summary')

data = col_raw.get()

print(data.keys())
len(data['documents'])

dict_keys(['ids', 'embeddings', 'metadatas', 'documents', 'uris', 'data', 'included'])


3180

In [35]:
results = col_raw.query(query_texts="what is graph rag?",n_results=5)
print(results.keys())
results['documents']


dict_keys(['ids', 'distances', 'metadatas', 'embeddings', 'documents', 'uris', 'data', 'included'])


[['9.4 GraphRAG in Industry\n\nIn this section, we mainly focus on industrial GraphRAG systems. These systems are characterized by their reliance on industrial graph database systems or their focus on large-scale graph data, details of which are as follows.\n\n• GraphRAG (by Microsoft)10: The system uses LLMs to construct entity-based knowledge graphs and pre-generate community summaries of related entity groups, which enables the capture of both local and global relationships within a document collection, thereby enhancing Query-Focused Summarization (QFS) task [32]. The project can also utilize open-source RAG toolkits for rapid implementation, such as LlamaIndex11, LangChain12, etc.\n\n• GraphRAG (by NebulaGraph)13: The project is the first industrial GraphRAG system, which is developed by NebulaGraph Corporation. The project integrates LLMs into the NebulaGraph database, which aims to deliver more intelligent and precise search results.\n\n• GraphRAG (by Antgroup)14: The framework 